In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
import os
import urllib
from urllib.request import urlopen
import pickle
import sys
import numpy as np
import pandas as pd
import elementpath
import re

In [5]:
# Define the user agents to rotate
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.79 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'
]


## scrape Putin's statements 2004-2023

In [12]:
# test on one page

base_url = 'http://kremlin.ru'
path = '/events/president/transcripts/statements/page/1'

# Make a GET request to the desired URL with rotating user agents and sleep time
for i in range(len(user_agents)):
    headers = {'User-Agent': user_agents[i]}
    url = base_url + path
    response = requests.get(url, headers=headers)

    # Check the status code
    if response.status_code == 200:
        # Parse the HTML content with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

# Sleep for a random time between 1 and 5 seconds before making the next request with a different user agent
    time.sleep(1 + 4 * random.random())
    i = (i + 1) % len(user_agents)
    headers = {'User-Agent': user_agents[i]}


In [14]:
for link in soup.find_all('a'):
    print(link.get('href'))


#sidebar
/
/events/president/news
/structure/president
/multimedia/video
/acts/news
/contacts
/search
None
/acts/bank
#
#clear
#
#clear
http://pravo.gov.ru/
https://www.may9.ru/
/events/president/news
/events/administration
/events/state-council
/events/security-council
/events/councils
#focus
/events/president/news
/events/president/transcripts
/events/president/trips
/events/president/letters
/structure/president
/events/president/transcripts
/events/president/transcripts/messages
/events/president/transcripts/statements
/events/president/transcripts/deliberations
/events/president/transcripts/speeches
/events/president/transcripts/comminity_meetings
/events/president/transcripts/press_conferences
/events/president/transcripts/interviews
/events/president/transcripts/articles
/events/president/transcripts/statements/70565
/events/president/transcripts/statements/70565
/events/president/transcripts/statements/70565/photos
/events/president/transcripts/statements/70565/videos
/events/p

In [16]:
soup.find_all('a')[29].get('href')


'/events/president/transcripts/statements'

In [20]:
# Find all links with an href attribute that starts with "/events/president/transcripts/messages"
statements_links = soup.find_all("a", href=lambda href: href and href.startswith(
    "/events/president/transcripts/statements"))

In [28]:
base_url = 'http://kremlin.ru'

# Find all links with an href attribute that starts with "/events/president/transcripts/messages"
statements_links = soup.find_all("a", href=lambda href: href and href.startswith(
    "/events/president/transcripts/statements"))

# convert the messages_links to a set to remove duplicates
statements_links = set(statements_links)

# create an empty list to store the urls
urls = []

# loop through the messages_links to create the urls
for link in statements_links:
    parts = link["href"].split("/")
    last_part = parts[-1]
    if last_part.isdigit():
        url = base_url + link["href"]
        urls.append(url)

urls = set(urls)
for url in urls:
    print(url)


http://kremlin.ru/events/president/transcripts/statements/67055
http://kremlin.ru/events/president/transcripts/statements/70159
http://kremlin.ru/events/president/transcripts/statements/68669
http://kremlin.ru/events/president/transcripts/statements/66975
http://kremlin.ru/events/president/transcripts/statements/69695
http://kremlin.ru/events/president/transcripts/statements/67203
http://kremlin.ru/events/president/transcripts/statements/66905
http://kremlin.ru/events/president/transcripts/statements/67438
http://kremlin.ru/events/president/transcripts/statements/68366
http://kremlin.ru/events/president/transcripts/statements/67040
http://kremlin.ru/events/president/transcripts/statements/70565
http://kremlin.ru/events/president/transcripts/statements/page/2
http://kremlin.ru/events/president/transcripts/statements/67551
http://kremlin.ru/events/president/transcripts/statements/69299
http://kremlin.ru/events/president/transcripts/statements/67044
http://kremlin.ru/events/president/tran

In [29]:
len(urls)

21

### Loop over 27 pages and get the urls

In [30]:
base_url = 'http://kremlin.ru'
path = '/events/president/transcripts/statements/page/'

In [31]:
urls = []

# Loop over 27 pages
for page_num in range(1, 28):
    url = base_url + path + str(page_num)
    headers = {'User-Agent': user_agents[page_num % len(user_agents)]}
    response = requests.get(url, headers=headers)

    # Check the status code
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        # TODO: Scrape the URLs from the page and save them to a list
        for link in soup.find_all('a'):
            href = link.get('href')
            if href and href.startswith('/'):
                urls.append(base_url + href)
                print(base_url + href)  # Print the URL while it works
    else:
        print(
            f'Request for {url} failed with status code {response.status_code}')

    # Sleep for a random time between 1 and 5 seconds before making the next request with a different user agent
    time.sleep(1 + 4 * random.random())


http://kremlin.ru/
http://kremlin.ru/events/president/news
http://kremlin.ru/structure/president
http://kremlin.ru/multimedia/video
http://kremlin.ru/acts/news
http://kremlin.ru/contacts
http://kremlin.ru/search
http://kremlin.ru/acts/bank
http://kremlin.ru/events/president/news
http://kremlin.ru/events/administration
http://kremlin.ru/events/state-council
http://kremlin.ru/events/security-council
http://kremlin.ru/events/councils
http://kremlin.ru/events/president/news
http://kremlin.ru/events/president/transcripts
http://kremlin.ru/events/president/trips
http://kremlin.ru/events/president/letters
http://kremlin.ru/structure/president
http://kremlin.ru/events/president/transcripts
http://kremlin.ru/events/president/transcripts/messages
http://kremlin.ru/events/president/transcripts/statements
http://kremlin.ru/events/president/transcripts/deliberations
http://kremlin.ru/events/president/transcripts/speeches
http://kremlin.ru/events/president/transcripts/comminity_meetings
http://kreml

In [32]:
len(urls)

3262

In [33]:
# filter only the statements + digits
filtered_urls = [url for url in urls if url.startswith(
    'http://kremlin.ru/events/president/transcripts/statements') and re.search(r'\d$', url)]


In [38]:
filtered_urls = set(filtered_urls)
for url in filtered_urls:
    print(url)


http://kremlin.ru/events/president/transcripts/statements/7265
http://kremlin.ru/events/president/transcripts/statements/page/18
http://kremlin.ru/events/president/transcripts/statements/14834
http://kremlin.ru/events/president/transcripts/statements/51155
http://kremlin.ru/events/president/transcripts/statements/4358
http://kremlin.ru/events/president/transcripts/statements/1360
http://kremlin.ru/events/president/transcripts/statements/60795
http://kremlin.ru/events/president/transcripts/statements/22338
http://kremlin.ru/events/president/transcripts/statements/60961
http://kremlin.ru/events/president/transcripts/statements/13605
http://kremlin.ru/events/president/transcripts/statements/8410
http://kremlin.ru/events/president/transcripts/statements/58037
http://kremlin.ru/events/president/transcripts/statements/63329
http://kremlin.ru/events/president/transcripts/statements/15387
http://kremlin.ru/events/president/transcripts/statements/7885
http://kremlin.ru/events/president/transcri

In [39]:
len(filtered_urls)

567

In [40]:
urls = list(filtered_urls)
len(urls)

567

In [71]:
# Save the list to a file
with open('data/corpus_adriana/corpus_president_of_russia/statements_urls.pkl', 'wb') as f:
    pickle.dump(urls, f)

# Load the list from the file
with open('data/corpus_adriana/corpus_president_of_russia/statements_urls.pkl', 'rb') as f:
    urls = pickle.load(f)


### Make a soup of statements_urls

In [42]:
# make a soup for each url of 567 urls object

soup_list = []
processed_urls = []

for url in urls:
    # Preprocess the URL
    processed_url = url.strip()

    # Check if the URL has already been processed
    if processed_url in processed_urls:
        print(f"Skipping {processed_url} - already processed")
        continue

    try:
        # Make a GET request to the desired URL with rotating user agents and sleep time
        i = 0
        while True:
            headers = {'User-Agent': user_agents[i]}
            response = requests.get(processed_url, headers=headers, timeout=10)

            if response.status_code == 200:
                # make a soup object from the response content
                soup = BeautifulSoup(response.content, 'html.parser')
                soup_list.append(soup)

                # Sleep for a random time between 1 and 5 seconds before making the next request with a different user agent
                time.sleep(1 + 4 * random.random())
                i = (i + 1) % len(user_agents)
                processed_urls.append(processed_url)
                print(f"Processed URL: {processed_url}")
                break

            else:
                # If the response is not 200, sleep for a random time between 1 and 5 seconds and try again with a different user agent
                time.sleep(1 + 4 * random.random())
                i = (i + 1) % len(user_agents)

    except Exception as e:
        print(f"Error getting data from {processed_url}: {e}")


Processed URL: http://kremlin.ru/events/president/transcripts/statements/7265
Processed URL: http://kremlin.ru/events/president/transcripts/statements/page/18
Processed URL: http://kremlin.ru/events/president/transcripts/statements/14834
Processed URL: http://kremlin.ru/events/president/transcripts/statements/51155
Processed URL: http://kremlin.ru/events/president/transcripts/statements/4358
Processed URL: http://kremlin.ru/events/president/transcripts/statements/1360
Processed URL: http://kremlin.ru/events/president/transcripts/statements/60795
Processed URL: http://kremlin.ru/events/president/transcripts/statements/22338
Processed URL: http://kremlin.ru/events/president/transcripts/statements/60961
Processed URL: http://kremlin.ru/events/president/transcripts/statements/13605
Processed URL: http://kremlin.ru/events/president/transcripts/statements/8410
Processed URL: http://kremlin.ru/events/president/transcripts/statements/58037
Processed URL: http://kremlin.ru/events/president/tran

In [45]:
# increase the recursion limit
sys.setrecursionlimit(10**6)

In [46]:
# Save the soup_list to a pickle file
with open('data/corpus_adriana/corpus_president_of_russia/soup_list.pkl', 'wb') as f:
    pickle.dump(soup_list, f)

In [2]:
# Load the soup_list from the pickle file
with open('data/corpus_adriana/corpus_president_of_russia/soup_list.pkl', 'rb') as f:
    soup_list = pickle.load(f)

In [3]:
len(soup_list)

567

In [32]:
for soup in soup_list:
    link_tag = soup.find('link', rel='alternate',
                         type='text/html', hreflang='en')
    if link_tag:
        print(link_tag['href'])
    else:
        print('URL not found')


http://en.kremlin.ru/events/president/transcripts/statements/7265
http://en.kremlin.ru/events/president/transcripts/statements/page/18
http://en.kremlin.ru/events/president/transcripts/statements/14834
http://en.kremlin.ru/events/president/transcripts/statements/51155
http://en.kremlin.ru/events/president/transcripts/statements/4358
http://en.kremlin.ru/events/president/transcripts/statements/1360
http://en.kremlin.ru/events/president/transcripts/statements/60795
http://en.kremlin.ru/events/president/transcripts/statements/22338
http://en.kremlin.ru/events/president/transcripts/statements/60961
http://en.kremlin.ru/events/president/transcripts/statements/13605
http://en.kremlin.ru/events/president/transcripts/statements/8410
http://en.kremlin.ru/events/president/transcripts/statements/58037
http://en.kremlin.ru/events/president/transcripts/statements/63329
http://en.kremlin.ru/events/president/transcripts/statements/15387
http://en.kremlin.ru/events/president/transcripts/statements/788

# filter out the url ending in page {digit}
filtered_soup_list = [soup for soup, url in zip(
    soup_list, urls) if not re.search(r'page/\d+$', url)]
len(filtered_soup_list)


In [51]:
# Save the filtered_soup_list to a pickle file
with open('data/corpus_adriana/corpus_president_of_russia/filtered_soup_list.pkl', 'wb') as f:
    pickle.dump(filtered_soup_list, f)


### load filtered_soup_list

In [2]:
# Load the soup_list from the pickle file
with open('data/corpus_adriana/corpus_president_of_russia/statements_filtered_soup_list.pkl', 'rb') as f:
    filtered_soup_list = pickle.load(f)


### find elements in each soup of filtered_soup_list

In [42]:
sorted_soup_list = sorted(filtered_soup_list, key=lambda soup: soup.find(
    'time', {'itemprop': 'datePublished'})['datetime'])

for soup in sorted_soup_list:
    datetime = soup.find('time', {'itemprop': 'datePublished'})['datetime']
    url = soup.find('div', {'class': 'share_link share_link_p'}).span.text.strip()
    print(f"{datetime} {url}")


2003-10-06 http://kremlin.ru/events/president/transcripts/statements/22147
2003-11-11 http://kremlin.ru/events/president/transcripts/statements/22197
2003-11-13 http://kremlin.ru/events/president/transcripts/statements/22204
2003-12-03 http://kremlin.ru/events/president/transcripts/statements/22237
2004-01-12 http://kremlin.ru/events/president/transcripts/statements/22279
2004-01-15 http://kremlin.ru/events/president/transcripts/statements/22281
2004-01-27 http://kremlin.ru/events/president/transcripts/statements/22338
2004-03-15 http://kremlin.ru/events/president/transcripts/statements/22384
2004-03-24 http://kremlin.ru/events/president/transcripts/statements/22394
2004-03-25 http://kremlin.ru/events/president/transcripts/statements/22395
2004-03-29 http://kremlin.ru/events/president/transcripts/statements/22401
2004-03-30 http://kremlin.ru/events/president/transcripts/statements/22403
2004-04-28 http://kremlin.ru/events/president/transcripts/statements/22442
2004-05-09 http://kremlin

In [34]:
len(url)

66

In [49]:
soup = filtered_soup_list[20]
b_tags = soup.find_all('b')
speakers = [b_tag.text.strip() for b_tag in b_tags]
print(speakers)


['В.Путин:', 'С.Саргсян     Саргсян Серж Азатович :', 'Вопрос:', 'В.Путин:', 'Вопрос:', 'В.Путин:']


In [36]:
for p in soup.find_all('p'):
    speaker_tag = p.find('b')
    if speaker_tag is not None:
        print(speaker_tag.text)

В.Путин:
С.Саргсян     Саргсян Серж Азатович :
Вопрос:
В.Путин:
Вопрос:
В.Путин: 


In [33]:
# Initialize a list to store the data
data = []

# Loop through all filtered soups
for soup in filtered_soup_list:
    # Extract the date and URL
    datetime = soup.find('time', {'itemprop': 'datePublished'})['datetime']
    url = soup.find('div', {'class': 'share_link share_link_p'}).span.text.strip()
    

    # Extract the title
    title = soup.find('h1', class_='entry-title').text.strip()

    # Find all <p> tags in the soup
    p_tags = soup.find_all('p')

    # Initialize the speaker variable to None
    speaker = None

    # Initialize a list to store the data for this document
    doc_data = []

    # Loop through all <p> tags and assign a speaker if needed
    for p_tag in p_tags:
        # Reset the speaker variable to None at the beginning of each <p> tag loop
        speaker = None

        # Check if the <p> tag has a <b> tag inside it
        if p_tag.find('b'):
            # If it does, save the <b> tag as the current speaker
            speaker = p_tag.find('b').text.strip()

        # Extract the text from the <p> tag (excluding the <b> tag if present)
        text = ' '.join(p_tag.find_all(string=True, recursive=False)).strip()
        if speaker:
            text = text.replace(speaker, '', 1).strip()

        # Add the data for this <p> tag to the list for this document
        doc_data.append({'date': datetime, 'url': url, 'title': title, 'speaker': speaker, 'text': text})

    # Add the data for this document to the overall list of data
    data.extend(doc_data)

# Create a pandas DataFrame from the data
df = pd.DataFrame(data)


In [40]:
# save the dataframe
#save the df as pickle
df.to_pickle('data/corpus_adriana/corpus_president_of_russia/statements_on_major_issues.pkl')

In [41]:
df = pd.read_pickle('data/corpus_adriana/corpus_president_of_russia/statements_on_major_issues.pkl')
df

,date,url,title,speaker,text
0,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,None,Все материалы сайта доступны по лицензии:
1,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,None,
2,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,None,В ходе специального совещания в связи с террор...
3,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,None,Глава государства поручил руководству правоохр...
4,2010-03-29,http://kremlin.ru/events/president/transcripts...,Совещание в связи с терактами в метро,None,Правительству поручено оказать необходимую пом...
...,...,...,...,...,...
67651,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,
67652,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,Официальный сайт президента России:
67653,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,Инвестиционный форум «Россия зовёт!»
67654,2019-11-20,http://kremlin.ru/events/president/transcripts...,Инвестиционный форум «Россия зовёт!»,В.Путин:,http://kremlin.ru/events/president/transcripts...
